In [11]:
%%writefile src/data_setup.py
import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader


def create_dataloaders(train_dir, 
                       test_dir, 
                       train_transform,
                       test_transform,
                       batch_size):
    
    train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
    test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)
    
    class_names = train_dataset.classes
    
    train_data_loader = DataLoader(
        train_dataset, 
        batch_size,
        shuffle=True,
        pin_memory=True
    )
    
    test_data_loader = DataLoader(
        test_dataset, 
        batch_size,
        shuffle=True,
        pin_memory=True
    )
    
    return train_data_loader, test_data_loader, class_names

Overwriting src/data_setup.py


In [8]:
%%writefile src/model_builder.py
import torch
from torch import nn

class ImageClassificationModel(nn.Module):
    
    def __init__(self, input_channels, hidden_channels, output_channels):
        
        super().__init__()
        
        self.layer_1 = nn.Sequential(
            nn.Conv2d(input_channels, hidden_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.layer_2 = nn.Sequential(
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=3, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        self.layer_3 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_channels*16*16, output_channels)
        )
     
    def forward(self, x):
        return self.layer_3(self.layer_2(self.layer_1(x)))

Overwriting src/model_builder.py


In [11]:
%%writefile src/engine.py
import torch

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc


def train_step(
    model, 
    data_loader,
    loss_fn,
    optimizer,
    device
):

    model.train()
    train_loss = 0
    train_acc = 0

    #Run through all the batches
    for batch, (X, y) in enumerate(data_loader):

        X = X.to(device)
        y = y.to(device)

        y_logits = model(X)

        loss = loss_fn(y_logits, y)
        train_loss = train_loss + loss

        acc = accuracy_fn(y, torch.argmax(y_logits, dim=1))
        train_acc = train_acc + acc

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

    train_loss = train_loss / len(data_loader)
    train_acc = train_acc / len(data_loader)

    return train_loss, train_acc


def test_step(
    model, 
    data_loader,
    loss_fn,
    device
):

    model.eval()
    test_loss = 0
    test_acc = 0

    with torch.inference_mode():
        #Run through all the batches
        for batch, (X, y) in enumerate(data_loader):

            X = X.to(device)
            y = y.to(device)

            y_logits = model(X)

            loss = loss_fn(y_logits, y)
            test_loss = test_loss + loss

            acc = accuracy_fn(y, torch.argmax(y_logits, dim=1))
            test_acc = test_acc + acc

        test_loss = test_loss / len(data_loader)
        test_acc = test_acc / len(data_loader)

    return test_loss, test_acc


def train(
    model, 
    train_data_loader,
    test_data_loader,
    loss_fn,
    optimizer,
    device,
    epochs
):
    
    results = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': []
    }
    
    for epoch in torch.arange(epochs):
    
        train_loss, train_acc = train_step(model, 
                                           train_data_loader, 
                                           loss_fn, 
                                           optimizer, 
                                           device)

        test_loss, test_acc = test_step(model, 
                                       test_data_loader, 
                                       loss_fn,
                                       device)
        
        print(
            f'Epoch: {epoch+1} | '
            f'train_loss: {train_loss} | '
            f'train_acc: {train_acc} | '
            f'test_loss: {test_loss} | '
            f'test_acc: {test_acc}'
        )
        
        results['train_loss']. append(train_loss)
        results['train_acc']. append(train_acc)
        results['test_loss']. append(test_loss)
        results['test_acc']. append(test_acc)
        
    return results

Overwriting src/engine.py


In [24]:
%%writefile src/utils.py
import torch
from pathlib import Path

def save_model(model,
               target_dir,
               model_name):


    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), 
               f=model_save_path)

Overwriting src/utils.py


In [ ]:
%%writefile src/train.py
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms

# Setup hyperparameters
NUM_EPOCHS = 20
BATCH_SIZE = 32
HIDDEN_UNITS = 20
LEARNING_RATE = 0.01

# Setup directories
train_dir = "data/pizza_steak_sushi/train"
test_dir = "data/pizza_steak_sushi/test"

# Setup target device
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

train_transform_trivial_augment = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor() 
])

# Create testing transform (no data augmentation)
test_transform_trivial_augment = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    train_transform=train_transform_trivial_augment,
    test_transform=test_transform_trivial_augment,
    batch_size=BATCH_SIZE
)

model = model_builder.ImageClassificationModel(
    input_channels=3,
    hidden_channels=HIDDEN_UNITS,
    output_channels=len(class_names)
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model,
             train_dataloader,
             test_dataloader,
             loss_fn,
             optimizer,
             device,
             NUM_EPOCHS)

Overwriting src/train.py


In [2]:
%run -i 'src/train.py'

/opt/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /opt/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Epoch: 1 | train_loss: 1.3551325798034668 | train_acc: 43.359375 | test_loss: 1.6381109952926636 | test_acc: 25.757575757575754
Epoch: 2 | train_loss: 1.140576958656311 | train_acc: 30.859375 | test_loss: 1.0977433919906616 | test_acc: 46.21212121212121
Epoch: 3 | train_loss: 1.1004348993301392 | train_acc: 29.6875 | test_loss: 1.0996538400650024 | test_acc: 30.018939393939394
Epoch: 4 | train_loss: 1.0964906215667725 | train_acc: 42.578125 | test_loss: 1.0941100120544434 | test_acc: 43.93939393939394
Epoch: 5 | train_loss: 1.102189302444458 | train_acc: 30.46875 | test_loss: 1.094565510749817 | test_acc: 37.973484848484844
Epoch: 6 | train_loss: 1.1003081798553467 | train_acc: 30.46875 | test_loss: 1.0993245840072632 | test_acc: 33.996212121212125
Epoch: 7 | train_loss: 1.1028800010681152 | train_acc: 30.46875 | test_loss: 1.1037713289260864 | test_acc: 30.018939393939394
Epoch: 8 | train_loss: 1.1017264127731323 | train_acc: 26.953125 | test_loss: 1.0999056100845337 | test_acc: 25.75